In [1]:
import pandas as pd


In [2]:
data_all = []
for i in range(4):
    data = pd.read_csv('data/combined_data_' + str(i+1)+'.txt', header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])
    data_all.append(data)

KeyboardInterrupt: 

In [ ]:
df = pd.concat(data_all)

In [ ]:
import numpy as np
df.index = np.arange(0,len(df))
print('Full dataset shape: {}'.format(df.shape))
print('-Dataset examples-')
print(df.iloc[::5000000, :])

Full dataset shape: (100498277, 2)
-Dataset examples-
           Cust_Id  Rating
0               1:     NaN
5000000    2560324     4.0
10000000   2271935     2.0
15000000   1921803     2.0
20000000   1933327     3.0
25000000   1465002     3.0
30000000    961023     4.0
35000000   1372532     5.0
40000000    854274     5.0
45000000    116334     3.0
50000000    768483     3.0
55000000   1331144     5.0
60000000   1609324     2.0
65000000   1699240     3.0
70000000   1776418     4.0
75000000   1643826     5.0
80000000    932047     4.0
85000000   2292868     4.0
90000000    932191     4.0
95000000   1815101     3.0
100000000   872339     4.0


In [ ]:
import numpy as np
df_nan = pd.DataFrame(pd.isnull(df.Rating))
df_nan = df_nan[df_nan['Rating'] == True]
df_nan = df_nan.reset_index()

movie_np = []
movie_id = 1

for i,j in zip(df_nan['index'][1:],df_nan['index'][:-1]):
    # numpy approach
    temp = np.full((1,np.abs(i-j-1)), movie_id)
    movie_np = np.append(movie_np, temp)
    movie_id += 1

# Account for last record and corresponding length
# numpy approach
last_record = np.full((1,len(df) - df_nan.iloc[-1, 0] - 1),movie_id)
movie_np = np.append(movie_np, last_record)

In [ ]:
df = df[pd.notnull(df['Rating'])]

df['Movie_Id'] = movie_np.astype(int)
df['Cust_Id'] = df['Cust_Id'].astype(int)
print('-Dataset examples-')
print(df.sample(3))

-Dataset examples-
          Cust_Id  Rating  Movie_Id
51388783  1560508     4.0      9298
17486820  1823426     5.0      3355
69922382  1322665     3.0     12672


In [3]:
df.to_csv('data/data_prep1.csv', sep=';', index = None)

NameError: name 'df' is not defined

In [4]:
import pandas as pd
df = pd.read_csv('data/data_prep1.csv', sep=';')

In [6]:
df.Movie_Id.nunique()

17770

In [7]:
df.Cust_Id.nunique()

480189

In [8]:
cust_sample = df.Cust_Id.sample(10000)

In [9]:
movie_sample = df.Movie_Id.sample(5000)

In [10]:
import surprise

In [11]:
from surprise import Reader, Dataset

In [12]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(df[df.Cust_Id.isin(cust_sample) &
                              df.Movie_Id.isin(movie_sample)][['Cust_Id', 'Movie_Id', 'Rating']], reader)

In [13]:
from surprise import KNNBasic

sim_options = {
    'name': 'cosine',
    'user_based': False
}
 
knn = KNNBasic(sim_options=sim_options)
trainingSet = data.build_full_trainset()

In [14]:
knn.fit(trainingSet)



Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
testSet = trainingSet.build_anti_testset()
predictions = knn.test(testSet)

In [ ]:
titles = pd.read_csv('data/movie_titles.csv', encoding = "ISO-8859-1", 
                     header = None, 
                     on_bad_lines='skip',
                     names = ['Movie_Id', 'Year', 'Name'])

In [ ]:
predictions[0:4]

[Prediction(uid=2477414, iid=17, r_ui=3.586179789789014, est=4.59980242801443, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=2477414, iid=18, r_ui=3.586179789789014, est=4.724826118541935, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=2477414, iid=19, r_ui=3.586179789789014, est=4.700345235427843, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=2477414, iid=28, r_ui=3.586179789789014, est=4.875274580788756, details={'actual_k': 40, 'was_impossible': False})]

In [ ]:
from collections import defaultdict
 
def get_top3_recommendations(predictions, topN = 3):
     
    top_recs = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_recs[uid].append((iid, est))
     
    for uid, user_ratings in top_recs.items():
        user_ratings.sort(key = lambda x: x[1], reverse = True)
        top_recs[uid] = user_ratings[:topN]
     
    return top_recs

In [ ]:
top3_recommendations = get_top3_recommendations(predictions)

In [ ]:
import numpy as np
def print_recs(i):
    for (a, b) in top3_recommendations[i]:
        print(titles[titles.Movie_Id == a]['Name'].values[0], np.round(b,2))

In [ ]:
i = 262149
print_recs(i)

In [ ]:
i = np.random.choice(list(top3_recommendations.keys()))

print_recs(i)

Band of Brothers 4.45
Schindler's List 4.4
Scarface: 20th Anniversary Edition 4.4


In [ ]:
films = data.df[(data.df.Cust_Id == i) & (data.df.Rating == 5)]['Movie_Id'].values
titles[titles.Movie_Id.isin(films)]['Name'].values

array(['The Phantom of the Opera: Special Edition', 'American Beauty',
       "Singin' in the Rain",
       'Lord of the Rings: The Fellowship of the Ring', 'Braveheart',
       'The King and I', 'Star Wars: Episode V: The Empire Strikes Back',
       'The Terminator', 'Lawrence of Arabia',
       'Lord of the Rings: The Two Towers: Extended Edition',
       'The Lord of the Rings: The Fellowship of the Ring: Extended Edition',
       'Minority Report', 'RoboCop', 'Terminator 2: Extreme Edition',
       'Caddyshack', 'Star Wars: Episode VI: Return of the Jedi',
       'Twister', 'Star Trek II: The Wrath of Khan',
       'Raiders of the Lost Ark', 'Forrest Gump',
       'Lord of the Rings: The Two Towers', 'Contact', 'Spider-Man 2',
       'The Godfather', "National Lampoon's Animal House", 'Men in Black',
       'Bicentennial Man', 'Blazing Saddles', 'Gladiator', 'Cast Away',
       'Lord of the Rings: The Return of the King', 'Jurassic Park',
       'The Shawshank Redemption: Special 

Задание 9.7.1

Укажите индекс непросмотренного пользователем №2 фильма, который имеет наибольшую оценку

In [ ]:
user_movie_matrix = np.array(((1,5,0,5,4), (5,4,4,3,2), (0,4,0,0,5), (4,4,1,4,0), (0,4,3,5,0), (2,4,3,5,3)))
user_movie_matrix.shape

(6, 5)

In [ ]:
from scipy import linalg  
U, s, V = linalg.svd(user_movie_matrix,full_matrices=False)
M=U @ np.diag(s) @ V # ок, работает.

In [ ]:
U_2 = U[2:, :]
V_2 = V[:2, :]

In [ ]:
user_2 = user_movie_matrix[1]

In [ ]:
user_2_reduced = user_2 @ V_2.T

In [ ]:
user_2_predicted = user_2_reduced @ V_2

In [ ]:
user_2_predicted 

array([3.19719556, 4.22057993, 2.91894245, 4.851947  , 1.05421441])

Задание 9.7.2

А теперь давайте проверим наше SVD разложение на новом пользователе. Выведите индекс непросмотренного новым пользователем фильма, который имеет наибольшую оценку. При этом новый пользователь что-то успел посмотреть: new_user = np.array((0, 0, 3, 4, 0))
Обратите внивание: SVD-разложение проводите без нового пользователя! В этот раз используйте 3 компоненты.

In [ ]:
new_user = np.array((0, 0, 3, 4, 0))

In [ ]:
user_new_reduced = new_user @ V_2.T

In [ ]:
user_new_predicted = user_new_reduced @ V_2
user_new_predicted

array([ 1.88933495,  1.46416086,  1.72840648,  2.3739358 , -0.58706807])

In [16]:
import pandas as pd
import numpy as np
import surprise
import apyori

In [17]:
data = surprise.Dataset.load_builtin("ml-100k")
trainingSet = data.build_full_trainset()

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\Ustenko-S/.surprise_data/ml-100k


In [18]:
testSet = trainingSet.build_testset()

Задание 9.10.1

Найдите количество пользователей в trainingSet

In [23]:
trainingSet.n_users

943

In [24]:
trainingSet.n_items

1682